In [1]:
!pip install datasets
!pip install nltk
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52563 sha256=510372efe46f1b9c26e1dbf1fc2cee69f2c94a4a9aadea8f9886010073768be7
  Stored in directory: /root/.cache/pip/wheels/c7/5e/28/47349211ec1f91379f41ed10bc2520f7071ecfb6cbe182f6fe
Successfully built polyglot
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.1/268.1 kB 3.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyicu: filename=pyicu-2.16-cp312-cp312-linux_x86_64.whl size=2721119 sha256=902c3088f73fdb13794c3c98a5cbbe2a89deee55933a86126b81889cd336d1dc
  Stored in directory: /root/.cache/pip/wheels/d8/d3/8e/efa19e1de2f9cf1451baf994a55dddcaa41a94185f2501ca08
Successfully built pyicu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 45.9 MB/s eta 0:00:0

In [2]:
!pip install stanza

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 40.9 MB/s eta 0:00:00


In [3]:
import nltk
nltk.download('punkt')

nltk.download('stopwords')

import stanza
# Stanza Türkçe modelini indir (bir kez indirmeniz yeterli)
stanza.download("tr")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: tr (Turkish) ...


INFO:stanza:Downloaded file to /root/stanza_resources/tr/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


In [4]:

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from datasets import load_dataset
from collections import Counter
from nltk.tokenize import word_tokenize
import re
from sklearn.metrics import accuracy_score
# NewsClassifier sınıfı
class NewsClassifier:
    def __init__(self, dataset_name="mcemilg/news-cat"):
        """
        Dataset'i yükler ve model için gerekli ön işlemleri yapar.
        """
        # Veri setini yükle
        self.dataset = load_dataset(dataset_name)
        self.model = None
        self.nlp = stanza.Pipeline("tr", processors="tokenize,pos,lemma")
        self.durak_kelimeler = { "ve", "veya", "bir", "bu", "şu", "çok", "az", "ise", "de", "da", "ile",
                                 "ama", "çünkü", "fakat", "ancak", "hem", "yine", "neden", "niçin", "şimdi",
                                 "o", "diğer", "bunun", "için", "her", "oldukça", "içinde", "olarak", "vardır"
                                 }

        # Eğitim ve test verilerini hazırlama
        self.train_data, self.test_data = self._prepare_data()
        self.keywords = {
            "spor": ["futbol", "basketbol", "tenis", "maç", "gol", "takım"],
            "ekonomi": ["ekonomi", "finans", "borsa", "piyasa", "yatırım", "ticaret","para"],
            "magazin": ["sanat", "ünlü", "dizi", "film", "şarkıcı", "aktör"],
            "politika": ["hükümet", "seçim", "meclis", "politik", "başkan", "parti"],
            "sağlık": ["sağlık", "hastane", "doktor", "tedavi", "ilaç", "hastalık"]
        }


    def _prepare_data(self):
        """
        Dataset'i eğitim ve test verisi olarak ayırır.
        """
        train_data = self.dataset['train']
        test_data = self.dataset['test']
        X_train = train_data['text']
        y_train = train_data['label']
        X_test = test_data['text']
        y_test = test_data['label']
        return (X_train, y_train), (X_test, y_test)

    def train(self):
        """
        Naive Bayes sınıflandırma modelini eğitir.
        """
        # Metinleri sayısal verilere dönüştürmek için bir pipeline oluştur
        self.model = make_pipeline(
            TfidfVectorizer(),
            MultinomialNB()
        )

        # Modeli eğit
        X_train, y_train = self.train_data
        self.model.fit(X_train, y_train)

    def predict(self, text):
        """
        Eğitilen model ile sınıflandırma yapar.
        """
        if self.model is None:
            print("Model eğitilmemiş!")
            return None
        return self.model.predict([text])[0]

# Sınıfa dinamik yöntem eklemek için dekoratör
def add_method_to_class(cls):
    def decorator(func):
        setattr(cls, func.__name__, func)
        return func
    return decorator

In [5]:
@add_method_to_class(NewsClassifier)
def evaluate(self):
    # Modeli değerlendir
    X_test, y_test = self.test_data
    predictions = self.model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    print(f"Model accuracy: {accuracy * 100:.2f}%")


In [6]:
@add_method_to_class(NewsClassifier)
def predict(self, text):
    # Yeni bir haber metnini sınıflandır
    prediction = self.model.predict([text])
    return prediction[0]

In [7]:
import string
nlp = stanza.Pipeline("tr", processors="tokenize,pos,lemma")
# Türkçe durak kelimeler listesi
durak_kelimeler = {
    "ve", "veya", "bir", "bu", "şu", "çok", "az", "ise", "de", "da", "ile",
    "ama", "çünkü", "fakat", "ancak", "hem", "yine", "neden", "niçin", "şimdi",
    "o", "diğer", "bunun", "için", "her", "oldukça", "içinde", "olarak", "vardır"
}

@add_method_to_class(NewsClassifier)
def tokenize_ve_kok_bul(self, metin):
    doc = nlp(metin)
    token_list = []
    for sentence in doc.sentences:
        for word in sentence.words:
            kelime = word.lemma  # Kelimenin kökünü al
            # Noktalama işaretlerini ve durak kelimeleri filtrele
            if kelime not in string.punctuation and kelime not in durak_kelimeler:
                token_list.append(kelime)
    return token_list




INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: tr (Turkish):
| Processor | Package       |
-----------------------------
| tokenize  | imst          |
| mwt       | imst          |
| pos       | imst_charlm   |
| lemma     | imst_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords

@add_method_to_class(NewsClassifier)
def extract_keywords(self, token_list, top_n=5):
    """
    Tokenlardan en sık kullanılan kelimeleri çıkarır.
    Durak kelimeler ve kelime köklemeyi dikkate alır.
    """
    # Durak kelimeleri filtrele
    stop_words = set(stopwords.words('turkish'))
    filtered_tokens = [word for word in token_list if word not in stop_words]

    # TF-IDF ile anahtar kelimeleri çıkar
    vectorizer = TfidfVectorizer(max_features=top_n)
    tfidf_matrix = vectorizer.fit_transform([' '.join(filtered_tokens)])

    # TF-IDF ağırlıklarına göre kelimeleri al
    features = vectorizer.get_feature_names_out()
    keywords = [features[i] for i in tfidf_matrix.sum(axis=0).argsort()[::-1]]

    return keywords



In [9]:
durak_kelimeler = { "ve", "veya", "bir", "bu", "şu", "çok", "az", "ise", "de", "da", "ile",
                                 "ama", "çünkü", "fakat", "ancak", "hem", "yine", "neden", "niçin", "şimdi",
                                 "o", "diğer", "bunun", "için", "her", "oldukça", "içinde", "olarak", "vardır"
                                 }
@add_method_to_class(NewsClassifier)
def ner_analysis(self, metin):
    # Stanza NER pipeline sonuçları
    doc = self.nlp(metin)

    kisiler = []
    kurumlar = []

    # Kişi ve kurumları daha doğru ayırt etme
    for sent in doc.sentences:
        temp_kisiler = []
        temp_kurumlar = []

        for word in sent.words:
            if word.upos == 'PROPN' and len(word.text) > 1:
                if word.text.istitle():  # Eğer baş harfleri büyükse kurum olabilir
                    temp_kurumlar.append(word.text)
                else:
                    temp_kisiler.append(word.text)

        # İsimleri ve kurumları ayırıp tek tek ekleyerek virgül ile ayırmak
        if temp_kisiler:
            kisiler.append(', '.join(temp_kisiler))  # Virgülle ayırarak ekle
        if temp_kurumlar:
            kurumlar.append(', '.join(temp_kurumlar))  # Virgülle ayırarak ekle

    # Filtreleme işlemi ve durak kelimelerden arındırma (Tekrar eden 'Ankara' gibi kelimeleri kaldır)
    kisiler = [kisi for kisi in kisiler if kisi.lower() not in self.durak_kelimeler]
    kurumlar = [kurum for kurum in kurumlar if kurum.lower() not in self.durak_kelimeler]

    # Sonuçları daha anlamlı bir şekilde döndürme
    return ', '.join(kisiler), ', '.join(kurumlar)




In [10]:


@add_method_to_class(NewsClassifier)
def topic_density_analysis(self, token_list):
    """
    Tokenlara ayrılmış bir metinde konu yoğunluğunu analiz eder.
    """
    # Kelime sıklıklarını hesapla
    word_counts = Counter(token_list)
    topic_density = {}
    total_words = sum(word_counts.values())

    for topic, key_list in self.keywords.items():
        topic_count = sum(word_counts[key] for key in key_list if key in word_counts)
        topic_density[topic] = (topic_count / total_words) * 100 if total_words > 0 else 0

    return topic_density


In [11]:
# Kullanım örneği
if __name__ == "__main__":
    # Modeli oluştur ve eğit
    news_classifier = NewsClassifier()
    news_classifier.train()

    # Modelin doğruluğunu değerlendir
    news_classifier.evaluate()

    # Etkileşimli test: Kullanıcıdan bir haber metni al ve sınıflandır
    while True:
        user_input = input("Bir haber metni girin (çıkmak için 'exit' yazın): ")
        if user_input.lower() == 'exit':
            break

        # Sınıflandırma
        label = news_classifier.predict(user_input)
        print(f"Sınıflandırılan etiket: {label}")

        token_list = news_classifier.tokenize_ve_kok_bul(user_input)  # Tokenize and preprocess the text

        # Anahtar Kelime Çıkarımı
        keywords = news_classifier.extract_keywords(token_list)  # Anahtar kelimeleri oluştur

       # Convert numpy arrays to strings
        keywords = [str(keyword) for keyword in keywords]
        print("Anahtar Kelimeler:", ", ".join(keywords))


        # NER Analizi
        kisiler, kurumlar = news_classifier.ner_analysis(user_input)
        print("Önemli kişiler ve kurumlar:", kisiler,kurumlar)


        # Konu Yoğunluğu Analizi

        topic_density = news_classifier.topic_density_analysis(token_list)
        print("Konu Yoğunluğu Analizi:")
        for topic, density in topic_density.items():
            print(f"{topic.capitalize()}: {density:.2f}%")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/125 [00:00<?, ?B/s]

train.jsonl: 0.00B [00:00, ?B/s]

val.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

...

Generating validation split:   0%|          | 0/150 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/250 [00:00<?, ? examples/s]

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: tr (Turkish):
| Processor | Package       |
-----------------------------
| tokenize  | imst          |
| mwt       | imst          |
| pos       | imst_charlm   |
| lemma     | imst_nocharlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Done loading processors!


Model accuracy: 93.20%
Bir haber metni girin (çıkmak için 'exit' yazın): galatasaray sutopu takimi nordic lig sampiyonu oldu
Sınıflandırılan etiket: sport
Anahtar Kelimeler: [['galatasaray' 'lig' 'nordic' 'ol' 'sampiyon']]
Önemli kişiler ve kurumlar: galatasaray, nordic 
Konu Yoğunluğu Analizi:
Spor: 0.00%
Ekonomi: 0.00%
Magazin: 0.00%
Politika: 0.00%
Sağlık: 0.00%
Bir haber metni girin (çıkmak için 'exit' yazın): exit
